In [1]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [2]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print(f"versja: {tf.__version__}")
print(f"eager mode: {tf.executing_eagerly()}")
print(f"hub - wersja: {hub.__version__}")
print("GPU jest","dostępne" if tf.config.list_physical_devices("GPU") else "nie dostępna!")

versja: 2.15.0
eager mode: True
hub - wersja: 0.16.1
GPU jest nie dostępna!


In [3]:
#pobranie danych
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]','train[:60%]','test'),
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKTPFHX/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKTPFHX/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKTPFHX/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [6]:
#tworzenie modelu
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding,input_shape=[],dtype=tf.string,trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [9]:
#budowa pełnego model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 16)                816       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy']
)

In [12]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1
                    )

Epoch 1/10
30/30 [==============================] - 78s 2s/step - loss: 0.6223 - accuracy: 0.5931 - val_loss: 0.5384 - val_accuracy: 0.6916
Epoch 2/10
30/30 [==============================] - 77s 3s/step - loss: 0.4801 - accuracy: 0.7635 - val_loss: 0.4026 - val_accuracy: 0.8113
Epoch 3/10
30/30 [==============================] - 64s 2s/step - loss: 0.3565 - accuracy: 0.8510 - val_loss: 0.2927 - val_accuracy: 0.8892
Epoch 4/10
30/30 [==============================] - 58s 2s/step - loss: 0.2631 - accuracy: 0.9025 - val_loss: 0.2176 - val_accuracy: 0.9339
Epoch 5/10
30/30 [==============================] - 61s 2s/step - loss: 0.1971 - accuracy: 0.9345 - val_loss: 0.1633 - val_accuracy: 0.9412
Epoch 6/10
30/30 [==============================] - 60s 2s/step - loss: 0.1487 - accuracy: 0.9553 - val_loss: 0.1184 - val_accuracy: 0.9683
Epoch 7/10
30/30 [==============================] - 61s 2s/step - loss: 0.1091 - accuracy: 0.9705 - val_loss: 0.0843 - val_accuracy: 0.9787
Epoch 8/10
30/30 [==

In [15]:
results = model.evaluate(test_data.batch(512),verbose=2)
for name,value in zip(model.metrics_names,results):
  print(f"{name}: {value}")

49/49 - 15s - loss: 0.3681 - accuracy: 0.8518 - 15s/epoch - 301ms/step
loss: 0.3681449294090271
accuracy: 0.8518400192260742
